In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
or_tract = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_41_tract.zip")

In [3]:
bike_counts_loc = gpd.read_file(r'T:\DCProjects\Modeling\AADBT\input\shp\BikeCountsLocations.shp')

In [6]:
census_vars = pd.read_csv(r'T:\DCProjects\Modeling\AADBT\input\census_vars.csv')

In [7]:
print(census_vars.head(2))

   TotalPop    Male  Female        GEOID  Year   Youth   Adult  Senior  \
0    5065.0  2447.0  2618.0  41039001302  2012  1510.0  2844.0   765.0   
1    4944.0  2477.0  2467.0  41039001600  2012  1064.0  3130.0   857.0   

    PctMale  PctFemale  PctMinor    PctEmp    NoHH  HHMedInc  HHMeaInc  \
0  0.483119   0.516881  0.136426  0.861722  1819.0   39455.0   46805.0   
1  0.501011   0.498989  0.097694  0.883519  1884.0   54505.0   63775.0   

   PctBikeCom  Pct30MinCom  
0    0.011327     0.579710  
1    0.000000     0.429171  


In [8]:
print(or_tract.head(2))

  STATEFP COUNTYFP TRACTCE        GEOID    NAME             NAMELSAD  MTFCC  \
0      41      019  100000  41019100000    1000    Census Tract 1000  G5020   
1      41      053  020306  41053020306  203.06  Census Tract 203.06  G5020   

  FUNCSTAT       ALAND    AWATER     INTPTLAT      INTPTLON  \
0        S  2972506997  16731802  +43.3251132  -122.5428476   
1        S   220829030   1880612  +44.7954486  -123.2452024   

                                            geometry  
0  POLYGON ((-123.20257 43.31449, -123.20254 43.3...  
1  POLYGON ((-123.36479 44.83828, -123.36469 44.8...  


In [15]:
or_tract.columns

Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAME', 'NAMELSAD', 'MTFCC',
       'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry'],
      dtype='object')

In [17]:
or_tract = or_tract[['GEOID', 'ALAND', 'geometry']]

In [65]:
census_vars.loc[:, 'GEOID'] = census_vars.GEOID.astype(str).values

In [72]:
lane_census = or_tract.merge(census_vars, on='GEOID')

In [74]:
lane_census = lane_census[lane_census.Year == 2020]

In [63]:
# points in polygons
def get_pip(points, polygon):
    id_list = list(polygon.GEOID)
    df = pd.DataFrame().reindex_like(points).dropna()
    for ID in id_list:
        pol = (polygon.loc[polygon.GEOID==ID])
        pol.reset_index(drop = True, inplace = True)
        pip_mask = points.within(pol.loc[0, 'geometry'])
        pip_data = points.loc[pip_mask].copy()
        pip_data['GEOID']= ID
        df = df.append(pip_data)
    df.reset_index(inplace=True, drop=True)
    df = df.drop(columns='geometry')
    return df

In [69]:
bike_counts = pd.read_csv(r'T:\DCProjects\Modeling\AADBT\input\Daily_Bike_Counts_With_VarData.csv')

In [71]:
bike_counts.columns

Index(['Location', 'Date', 'DailyCounts', 'Month', 'MonthDesc', 'Season',
       'Weekday', 'IsHoliday', 'UoInSession', 'IsSpecialEvent', 'CountType',
       'Direction', 'FacilityType', 'RoadWidth', 'City', 'Latitude',
       'Longitude', 'Site_Name', 'DoubleCountLocation', 'IsOneway',
       'OnewayDirection', 'IsSidewalk', 'Location_Description', 'PRCP', 'SNOW',
       'TMAX', 'Daylight_Mins', 'LineType', 'PM25'],
      dtype='object')

In [64]:
df = get_pip(points = bike_counts_loc, polygon = lane_census)